# TP1 - Grupo 14

André Lucena Ribas Ferreira - A94956

Paulo André Alegre Pinto - A97391

## Problema 1 - Alocação de Horário de uma StartUp

Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:

1. Cada reunião ocupa uma sala (enumeradas 1...S) durante um “slot”  1..T (hora,dia).  
2. Cada reunião tem associado um projeto (enumerados 1..P) e um conjunto de participantes. Os diferentes colaboradores são enumerados 1..C.
3. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. 
4. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  50 do total de colaboradores do projeto. 

Como 'inputs', o problema recebe:
1. O número de 'Timeslots' $T$;
2. O número de Salas $S$;
3. O número de Projetos $P$;
4. O número de Colaboradores $C$;
5. O conjunto de Colaboradores de cada Projeto, o seu Líder e o número mínimo de reuniões semanais, num dicionário $proj$.
6. A disponibilidade de cada Colaborador. Essa disponibilidade é representada numa matriz booleana de acessibilidade $disp$.

## Análise

Pretende-se alocar reuniões de Projetos a certas Salas, dentro de determinados 'Timeslots', aos Colaboradores de uma StartUp.

Para tal decidimos usar uma família de variáveis binárias $x_{t,s,p,c}$ definidas com a seguinte semântica:

$$ x_{t,s,p,c} == 1 \quad \mbox{se e só se} \quad \mbox{o Colaborador $c$ estará Presente numa reunião do Projeto $p$ na Sala $s$ no 'Timeslot' $t$} $$

O problema dispõe de limitações que deverão ser tratadas:

1. Para cada 'Timeslot', cada Líder tem de estar Presente nas reuniões do seu próprio Projeto;
2. Para cada 'Timeslot', cada Colaborador está Presente, no máximo, numa reunião;
3. Para cada 'Timeslot', cada Colaborador está Presente numa reunião só se pertencer a esse Projeto;
4. Para cada 'Timeslot', cada Colaborador está Presente numa reunião só se estiver disponível nesse 'Timeslot';
5. Para cada 'Timeslot', cada Sala tem a reunião de, no máximo, um Projeto;
6. Para cada 'Timeslot', numa dada Sala, cada Projeto pode ter reunião só se houver 'quórum';
7. Para cada Projeto, o número das suas reuniões tem de ser maior ou igual ao mínimo descrito.

Do mesmo modo, o problema admite certas optimizações:
1. Maximizar o número de reuniões efetivamente realizadas;
2. Minimizar o número médio de reuniões por participante.

## Implementação

Para a resolução do problema em questão, utilizaremos a biblioteca de programação linear do OR-Tools, o pywraplp. Portanto, começaremos por instalar o OR-Tools, importar tal biblioteca e inicializar o solver, que chamaremos de horario.

In [1]:
!pip install ortools

In [2]:
from ortools.linear_solver import pywraplp

## Problema 1 - Horário

Definimos o seguinte exemplo introdutório com solução, com mais exemplos no final do relatório

In [3]:
T, S, P, C = 6, 2, 5, 8 

proj = {}
proj[0] = {'colaboradores':[0,1,2,4], 'lider':4, 'min':1}
proj[1] = {'colaboradores':[1,2,3], 'lider':1, 'min':2}
proj[2] = {'colaboradores':[0,2,3,4], 'lider':3, 'min':2}
proj[3] = {'colaboradores':[0,1,7], 'lider':7, 'min':3}
proj[4] = {'colaboradores':[2,3,4], 'lider':2, 'min':1}

disp = {}
disp[0] = [1,0,1,1,0,1]
disp[1] = [1,1,1,1,0,1]
disp[2] = [1,1,0,1,0,1]
disp[3] = [1,1,1,0,1,1]
disp[4] = [1,1,1,0,1,1]
disp[5] = [1,0,1,0,1,1]
disp[6] = [1,1,1,0,1,1]
disp[7] = [1,1,1,0,0,1]

Para a família de variáveis binárias $x_{t,s,p,c}$, construíremos uma matriz de alocação $X \subseteq \{0,1\}^{TxSxPxC}$, para representar as  variáveis binárias. Declaramos a matriz de alocação $X$ como um dicionário:

In [4]:
X = {}
horario = pywraplp.Solver.CreateSolver('SCIP')
#dicionário com T*P*C*S variáveis -> X[t][p][c][s] == 1 sse c estará presente na reunião do projeto p no timeslot t na sala s.
for t in range(T):
    X[t] = {}
    for s in range(S):
        X[t][s] = {}
        for p in range(P):
            if disp[proj[p]['lider']][t] == 1:
                X[t][s][p] = {}
                #Cada Colaborador apenas pode participar dos seus proj -> Obrigatório pela definição dos dicionários
                for c in proj[p]['colaboradores']:
                    #Cada Colaborador tem de ter horário -> Obrigatório pela definição dos dicionários
                    if disp[c][t] == 1:
                        X[t][s][p][c] = horario.BoolVar('x[%i][%i][%i}[%i]' % (t,s,p,c))

### Restrições

1. Para cada Timeslot, cada Colaborador está Presente na reunião de um Projeto numa Sala só se o seu Líder também estiver.

Nota: A notação utilizada refere-se a, por exemplo, um 'Timeslot' $t$ pertencente ao dicionário $X$. Com isto, ao longo de todo o relatório, pretendemos ilustrar que o 'Timeslot' $t$ pertence às chaves do dicionário $X$. Tal é análogo a todas as restantes ocorrências de notação de elementos a pertencerem a dicionários.

$$ \forall_{t \in X}, \forall_{s \in X[t]}, \forall_{p \in X[t][s]}, \forall_{c \in X[t][s][p]}, \quad x_{t,s,p,c} <= x_{t,s,p,l}, \quad l = proj[p]['lider'] $$

In [5]:
for t in X:
    for s in X[t]:
        for p in X[t][s]:
            for c in X[t][s][p]:
                horario.Add(X[t][s][p][c] <= X[t][s][p][proj[p]['lider']])

2. Para cada 'Timeslot', cada Colaborador está Presente, no máximo, numa reunião:

$$ \forall_{t \in X}, \forall_{c < C}, \sum_{s \in X[t], p \in X[t][s]} x_{t,s,p,c} \le 1, \quad se \ c \in X[t][s][p] $$

In [6]:
for t in X:
    for c in range(C):
        horario.Add(sum([X[t][s][p][c] for s in X[t] for p in X[t][s] if c in X[t][s][p]]) <= 1)

3. Para cada 'Timeslot', cada Colaborador está Presente numa reunião só se pertencer a esse Projeto;

$$\forall_{t \in X}, \forall_{s \in X[t]}, \forall_{p \in X[t][s]}, \forall_{c \in X[t][s][p]}, x_{t,s,p,c} == 1 \quad se \ e \ só \ se \ o \ colaborador \ c \ estiver \ no \ projeto \ p $$

De acordo com a implementação que escolhemos para a nossa matriz de valores binários, não é possível um Colaborador $c$ participar de uma reunião na Sala $s$ do Projeto $p$ num 'Timeslot' $t$ ao qual não pertence. Isso deve-se tanto ao facto de sempre se iterar sobre os elementos do dicionário como de não existirem entradas $X[t][s][p][c]$ para Colaboradores $c$ que não pertençam ao Projeto $p$.

4. Para cada 'Timeslot', cada Colaborador está Presente numa reunião só se estiver disponível nesse 'Timeslot';

$$\forall_{t \in X}, \forall_{s \in X}, \forall_{p \in X}, \forall_{c \in X}, x_{t,s,p,c} == 1 \quad se \ e \ só \ se \ o \ colaborador \ c \ tiver \ disponibilidade $$

De acordo com a implementação que escolhemos para a nossa matriz de valores binários, não é possível um Colaborador $c$ participar de uma reunião na Sala $s$ do Projeto $p$ num 'Timeslot' $t$ ao qual não tem disponibilidade. Isso deve-se tanto ao facto de sempre se iterar sobre os elementos do dicionário como de não existirem entradas $X[t][s][p][c]$ para Colaboradores sem disponibilidade.

5. Para cada 'Timeslot', cada Sala tem a reunião de, no máximo, um Projeto;

Para verificar se um dado Projeto tem ou não reunião num dado 'Timeslot', pode-se tirar partido da variável $x_{t,s,p,l}$, onde $l$ é o Líder do projeto $p$. Isto dá-se pelo facto que:

$$ x_{t,s,p,l} == 1 \quad \mbox{se e só se} \quad \mbox{o Projeto $p$ tem reunião no 'Timeslot' $t$ na Sala $s$.} $$

In [7]:
for t in X:
    for s in X[t]:
        horario.Add(sum([X[t][s][p][proj[p]['lider']] for p in X[t][s]]) <= 1)

6. Para cada 'Timeslot', numa dada Sala, cada Projeto pode ter reunião só se houver 'quórum';

Por 'quórum' entende-se estarem Presentes, pelo menos, metade dos Colaboradores associados a esse Projeto.

Como dito anteriormente, a variável $x_{t,s,p,l}$, com $l$ o lider do Projeto $p$, indica se esse Projeto tem reunião no 'Timeslot' $t$ e na Sala $s$. Como tal, podemos utilizá-la para garantir o 'quórum' apenas se a reunião ocorrer.

Para evitar cálculos de divisões que possam resultar em valores não inteiros, em vez de se considerar a restrição:

$$ \forall_{t \in X}, \forall_{s \in X[t]}, \forall_{p \in X[t][s]}, (\sum_{c \in X[t][s][p]} x_{t,s,p,c}) >= \frac{\mbox{número de colaboradores de $p$}}{2} * x_{t,s,p,l} $$

Considerar-se-á a seguinte restrição equivalente:

$$ \forall_{t \in X}, \forall_{s \in X[t]}, \forall_{p \in X[t][s]}, (2 * \sum_{c \in X[t][s][p]} x_{t,s,p,c}) >= (\mbox{número de colaboradores de $p$}) * x_{t,s,p,l} $$

In [8]:
for t in X:
    for s in X[t]:
        for p in X[t][s]:
            horario.Add(2*sum([X[t][s][p][c] for c in X[t][s][p]]) >= 
                       len(proj[p]['colaboradores']) * X[t][s][p][proj[p]['lider']])

7. Para cada Projeto, o número das suas reuniões tem de ser maior ou igual ao mínimo descrito.

Da mesma forma, $x_{t,s,p,l}$ com $l$ o líder do Projeto $p$ indica se o esse Projeto tem reunião no 'Timeslot' $t$ e na Sala $s$.

Seja $min$ o valor mínimo de reuniões semanais temos então:

$$ \forall_ {p < P}, \sum_{t \in X, s \in X[t], c \in X[t][s][p]} x_{t,s,p,l} \ge min, \quad se \ p \in X[t][s] $$

In [9]:
for p in range(P):
        horario.Add(sum([X[t][s][p][proj[p]['lider']] for t in X for s in X[t] if p in X[t][s]]) >= proj[p]['min'])

Pode-se então tentar construir um horário para o exemplo dado. Nesta altura, ainda não se encontra otimizado para as especificações do problema.

In [10]:
status = horario.Solve() 
if status == pywraplp.Solver.OPTIMAL:
    for t in X:
        print(f'Slot: {t}')
        for s in X[t]:
            for p in X[t][s]:
                if proj[p]['lider'] in X[t][s][p] and X[t][s][p][proj[p]['lider']].solution_value() == 1:
                    l = proj[p]['lider']
                    print(f'Projeto {p} de líder {l} na Sala {s}')
                    print('Colaboradores:', end = ' ')
                    for c in X[t][s][p]:
                        if X[t][s][p][c].solution_value() == 1:
                            print(f'{c}', end = ' ')
                    print('')
        print('')
else:
    print('Nao é possivel construir o hórario')

Slot: 0
Projeto 4 de líder 2 na Sala 0
Colaboradores: 2 4 
Projeto 1 de líder 1 na Sala 1
Colaboradores: 1 3 

Slot: 1
Projeto 4 de líder 2 na Sala 0
Colaboradores: 2 3 
Projeto 3 de líder 7 na Sala 1
Colaboradores: 1 7 

Slot: 2
Projeto 0 de líder 4 na Sala 0
Colaboradores: 0 4 
Projeto 3 de líder 7 na Sala 1
Colaboradores: 1 7 

Slot: 3
Projeto 1 de líder 1 na Sala 0
Colaboradores: 1 2 

Slot: 4
Projeto 2 de líder 3 na Sala 0
Colaboradores: 3 4 

Slot: 5
Projeto 2 de líder 3 na Sala 0
Colaboradores: 0 3 
Projeto 3 de líder 7 na Sala 1
Colaboradores: 1 7 



### Optimização

Os critérios de optimização para este problema são os seguintes:
1. Maximizar o número de reuniões efetivamente realizadas - função $r$;
2. Minimizar o número médio de reuniões por participante - função $m$.

Para 1. o somatório dos $x_{t,s,p,l}$, para $l$ o líder do Projeto $p$, é o número de reuniões realizadas.
Para 2. o número médio de reuniões por participante dá-se por:


### Optimização

Os critérios de optimização para este problema são os seguintes:
1. Maximizar o número de reuniões efetivamente realizadas - função $r$;
2. Minimizar o número médio de reuniões por participante - função $m$.

Para 1. o somatório dos $x_{t,s,p,l}$, para $l$ o líder do Projeto $p$, é o número de reuniões realizadas.
Para 2. o número médio de reuniões por participante dá-se por:

$$ \frac{\sum_{t \in X} \sum_{s \in X[t]} \sum_{p \in X[t][s]} \sum_{c \in X[t][s][p]} x_{t,s,p,c}}{C} $$

Como $C$ é um número constante, basta minimzar o numerador da expressão.

Desse modo:

\begin{align*}
    r \quad &:= \quad \sum_{t \in X} \sum_{s \in X[t]} \sum_{p \in X[t][s]} x_{t,s,p,l} \\
    m \quad &:= \quad \sum_{t \in X} \sum_{s \in X[t]} \sum_{p \in X[t][s]} \sum_{c \in X[t][s][p]} x_{t,s,p,c} \\
\end{align*}

Pode-se obter esta optimização através de uma única maximização, considerando a seguinte função a necessária para maximizar:

$$(1- \lambda) * r - \lambda * m$$

O parâmetro $\lambda$ pode ser ajustado para tender a optimização ora a favor de maximizar $r$ como de maximizar $m$, desde que $\lambda \in [0,1]$.

In [11]:
lambd = 0.5
horario.Maximize((1 - lambd) * sum([X[t][s][p][proj[p]['lider']] for t in X for s in X[t] for p in X[t][s]]) - 
                  lambd * sum([X[t][s][p][c] for t in X for s in X[t] for p in X[t][s] for c in X[t][s][p]]))

## Exemplos

Nesta secção, apresentamos mais exemplos de horários criados pela resolução do problema.

De modo a facilitar a testagem dos exemplos, na célula colapsada seguinte encontra-se a função $scheduleStartUp(T, S, P, C, proj, disp)$. Esta função compõe todos os passos da implementação.

In [13]:
def scheduleStartUp(T, S, P, C, proj, disp):
    X = {}
    result = dict()
    horario = pywraplp.Solver.CreateSolver('SCIP')
    #dicionário com T*P*C*S variáveis -> X[t][p][c][s] == 1 sse c estará presente na reunião do projeto p no timeslot t na sala s.
    for t in range(T):
        X[t] = {}
        for s in range(S):
            X[t][s] = {}
            for p in range(P):
                if disp[proj[p]['lider']][t] == 1:
                    X[t][s][p] = {}
                    for c in proj[p]['colaboradores']:
                        if disp[c][t] == 1:
                            X[t][s][p][c] = horario.BoolVar('x[%i][%i][%i}[%i]' % (t,s,p,c))                
#Para cada Timeslot,
for t in X:
    #Cada Sala apenas pode ter um projeto
    for s in X[t]:
        horario.Add(sum([X[t][s][p][proj[p]['lider']] for p in X[t][s]]) <= 1)
            
    #Cada Colaborador possível nesse timeslot pode ter, no máximo, 1 reunião
    for c in range(C):
        horario.Add(sum([X[t][s][p][c] for s in X[t] for p in X[t][s] if c in X[t][s][p]]) <= 1)
            
    #Cada Colaborador tem de ter horário -> Obrigatório pela definição dos dicionários
    #Cada Colaborador apenas pode participar dos seus proj -> Obrigatório pela definição dos dicionários
        
    #Cada Líder tem de participar do seu projeto OU SEJA
        #Cada Colaborador participa apenas de um projeto se o seu líder participar
    for s in X[t]:
        for p in X[t][s]:
            for c in X[t][s][p]:
                horario.Add(X[t][s][p][c] <= X[t][s][p][proj[p]['lider']])
                    
     #Cada Projeto apenas pode ser selecionado se houver 'quórum', o projeto só existe se o líder existe
        #Apenas se pode olhar para uma dada sala, ou podemos ver salas de outros proj para algum colaborador
        #Número de participantes efetivos / total de colaboradores >= 1 / 2
            #Com cálculos chega-se a: 2*Número de participantes efetivos >= total de colaboradores
        #É necessário multiplicar pelo valor do líder para que não obriguemos a ter um somatório de 0's (pessoas não vão) >= quorum
    for s in X[t]:
        for p in X[t][s]:
            horario.Add(2*sum([X[t][s][p][c] for c in X[t][s][p]]) >= 
                        len(proj[p]['colaboradores']) * X[t][s][p][proj[p]['lider']])
        
#Para cada Projeto, o número de vezes que foi selecionado tem de ser igual ou maior ao seu mínimo
for p in range(P):
    horario.Add(sum([X[t][s][p][proj[p]['lider']] for t in X for s in X[t] if p in X[t][s]]) >= proj[p]['min'])
    
#Deve-se Maximizar o Número total de reuniões realizadas e Minimizar o número médio de reuniões por participante
lambd = 0.5
horario.Maximize((1 - lambd) * sum([X[t][s][p][proj[p]['lider']] for t in X for s in X[t] for p in X[t][s]]) - 
                    lambd * sum([X[t][s][p][c] for t in X for s in X[t] for p in X[t][s] for c in X[t][s][p]]))
    
status = horario.Solve()
    
if status == pywraplp.Solver.OPTIMAL:
    count = 0
    projects = 0
    for t in X:
        print(f'Slot: {t}')
        for s in X[t]:
            for p in X[t][s]:
                if proj[p]['lider'] in X[t][s][p] and X[t][s][p][proj[p]['lider']].solution_value() == 1:
                    projects += 1
                    l = proj[p]['lider']
                    print(f'Projeto {p} de líder {l} na Sala {s}')
                    print('Colaboradores:', end = ' ')
                    for c in X[t][s][p]:
                        if X[t][s][p][c].solution_value() == 1:
                            print(f'{c}', end = ' ')
                            count += 1
                    print('')
        print('')
    print(f'Número médio de reuniões por colaborador: {count/C}')
    print(f'Número de reuniões: {projects}')
else:
    print('Nao é possivel construir o hórario')

Slot: 0
Projeto 0 de líder 4 na Sala 0
Colaboradores: 1 4 
Projeto 4 de líder 2 na Sala 1
Colaboradores: 2 3 

Slot: 1
Projeto 3 de líder 7 na Sala 0
Colaboradores: 1 7 
Projeto 2 de líder 3 na Sala 1
Colaboradores: 2 3 

Slot: 2
Projeto 3 de líder 7 na Sala 0
Colaboradores: 1 7 
Projeto 2 de líder 3 na Sala 1
Colaboradores: 3 4 

Slot: 3
Projeto 1 de líder 1 na Sala 0
Colaboradores: 1 2 

Slot: 4

Slot: 5
Projeto 1 de líder 1 na Sala 0
Colaboradores: 1 3 
Projeto 3 de líder 7 na Sala 1
Colaboradores: 0 7 

Número médio de reuniões por colaborador: 2.25
Número de reuniões: 9


#### Exemplo 1 (Introdutório)

In [19]:
T, S, P, C = 6, 2, 5, 8 

proj = {}
proj[0] = {'colaboradores':[0,1,2,4], 'lider':4, 'min':1}
proj[1] = {'colaboradores':[1,2,3], 'lider':1, 'min':2}
proj[2] = {'colaboradores':[0,2,3,4], 'lider':3, 'min':2}
proj[3] = {'colaboradores':[0,1,7], 'lider':7, 'min':3}
proj[4] = {'colaboradores':[2,3,4], 'lider':2, 'min':1}

disp = {}
disp[0] = [1,0,1,1,0,1]
disp[1] = [1,1,1,1,0,1]
disp[2] = [1,1,0,1,0,1]
disp[3] = [1,1,1,0,1,1]
disp[4] = [1,1,1,0,1,1]
disp[5] = [1,0,1,0,1,1]
disp[6] = [1,1,1,0,1,1]
disp[7] = [1,1,1,0,0,1]

scheduleStartUp(T, S, P, C, proj, disp)

#### Exemplo 2 (impossível)

In [ ]:
T, S, P, C = 6, 2, 5, 8 

proj = {}
proj[0] = {'colaboradores':[0,1,2,4], 'lider':4, 'min':1}
proj[1] = {'colaboradores':[1,2,3], 'lider':1, 'min':2}
proj[2] = {'colaboradores':[0,2,3,4], 'lider':3, 'min':2}
proj[3] = {'colaboradores':[0,1,7], 'lider':7, 'min':3}
proj[4] = {'colaboradores':[2,3,4], 'lider':2, 'min':1}

disp = {}
disp[0] = [0,0,0,1,0,1]
disp[1] = [1,1,1,0,0,0]
disp[2] = [1,1,0,1,0,1]
disp[3] = [1,1,1,0,1,1]
disp[4] = [1,1,1,0,1,0]
disp[5] = [1,0,1,0,1,1]
disp[6] = [1,1,1,0,1,1]
disp[7] = [1,1,1,0,0,1]

#Não há disponibilidade dos colaboradores para se realizarem as reuniões do proj[0]

scheduleStartUp(T, S, P, C, proj, disp)

### Exemplo 3 (Possível)

In [ ]:
T, S, P, C = 6, 2, 5, 8 

proj = {}
proj[0] = {'colaboradores':[0,1,2,4], 'lider':4, 'min':1}
proj[1] = {'colaboradores':[1,2,3], 'lider':1, 'min':2}
proj[2] = {'colaboradores':[0,2,3,4], 'lider':3, 'min':2}
proj[3] = {'colaboradores':[0,1,7], 'lider':7, 'min':3}
proj[4] = {'colaboradores':[2,3,4], 'lider':2, 'min':1}

disp = {}
disp[0] = [0,0,0,1,0,1]
disp[1] = [1,1,1,0,0,0]
disp[2] = [1,1,0,1,0,1]
disp[3] = [1,1,1,0,1,1]
disp[4] = [1,1,1,0,1,0]
disp[5] = [1,0,1,0,1,1]
disp[6] = [1,1,1,0,1,1]
disp[7] = [1,1,1,0,0,1]
disp[8] = []
disp[9} = []

scheduleStartUp(T, S, P, C, proj, disp)

### Exemplo 4 (Possível)

In [23]:
T, S, P, C = 6, 2, 5, 8 

proj = {}
proj[0] = {'colaboradores':[0,1,2,3,4,5,6,7], 'lider':4, 'min':3}
proj[1] = {'colaboradores':[0,1,2,3,4,5,6,7], 'lider':1, 'min':3}

disp = {}
disp[0] = [1,1,1,1,1,1]
disp[1] = [1,1,1,1,1,1]
disp[2] = [1,1,1,1,1,1]
disp[3] = [1,1,1,1,1,1]
disp[4] = [1,1,1,1,1,1]
disp[5] = [1,1,1,1,1,1]
disp[6] = [1,1,1,1,1,1]
disp[7] = [1,1,1,1,1,1]

scheduleStartUp(T, S, P, C, proj, disp)

KeyError: 2

### Exemplo 5 (Possível)

In [ ]:
T, S, P, C = 6, 2, 5, 8 

proj = {}
proj[0] = {'colaboradores':[0,1,2,4,5,6,7], 'lider':4, 'min':3}
proj[1] = {'colaboradores':[1,2,3,4,5,6,7], 'lider':1, 'min':3}

disp = {}
disp[0] = [1,1,1,0,0,0]
disp[1] = [1,1,1,0,0,0]
disp[2] = [1,1,1,0,0,0]
disp[3] = [1,1,1,0,0,0]
disp[4] = [0,0,0,1,1,1]
disp[5] = [0,0,0,1,1,1]
disp[6] = [0,0,0,1,1,1]
disp[7] = [0,0,0,1,1,1]

#É possível por comparecer 50% dos colaboradores (com líder)

scheduleStartUp(T, S, P, C, proj, disp)

### Exemplo 6 (Impossível)

In [24]:
T, S, P, C = 6, 2, 5, 8 

proj = {}
proj[0] = {'colaboradores':[0,1,2,4,5,6,7], 'lider':4, 'min':3}
proj[1] = {'colaboradores':[1,2,3,4,5,6,7], 'lider':1, 'min':3}

disp = {}
disp[0] = [1,1,1,1,1,1]
disp[1] = [0,0,0,0,0,0]
disp[2] = [1,1,1,1,1,1]
disp[3] = [1,1,1,1,1,1]
disp[4] = [1,1,1,1,1,1]
disp[5] = [1,1,1,1,1,1]
disp[6] = [1,1,1,1,1,1]
disp[7] = [1,1,1,1,1,1]

# O colaborador 1 (líder do segundo projeto), não tem disponibilidade em nenhum dia.

scheduleStartUp(T, S, P, C, proj, disp)

KeyError: 2